In [3]:
import os 
import gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from nes_py.wrappers import JoypadSpace
import tensorboard as tf

# Função para criar o ambiente
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v1')
    env = JoypadSpace(env, RIGHT_ONLY)
    env = gym.wrappers.GrayScaleObservation(env, keep_dim=True)
    env = CustomRescaleObservation(env, low=0, high=255)
    env = gym.wrappers.FrameStack(env, num_stack=6)
    return env

# Classe personalizada para redimensionar as observações
class CustomRescaleObservation(gym.ObservationWrapper):
    def __init__(self, env, low, high):
        super(CustomRescaleObservation, self).__init__(env)
        self.low = low
        self.high = high

    def observation(self, observation):
        return observation * (self.high - self.low) + self.low

# Callback personalizado para recompensas e penalidades
class CustomCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(CustomCallback, self).__init__(verbose)
        self.x_pos = None
        self.check_freq = check_freq
        self.save_path = save_path
        self.rewards = 0
        self.time_before = None
        self.progress_bar = None

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        # Obtem informações sobre o ambiente após a etapa
        info = self.locals.get("info", {})

        # Recompensa o agente com base na diferença de x (v) entre os estados
        x_pos_after = info.get("x_pos", 0)
        self.x_pos = x_pos_after
        if self.x_pos is not None:
            v = x_pos_after - self.x_pos
            if v < 0:
                v = 1
            if v > 0:
                v = 2
            if v == 0:
                v = 0

        # Recompensa o agente com base na diferença no relógio (c) entre os estados
        time_after = info.get("time", 0)
        self.time_before = time_after
        if self.time_before is not None:
            c = self.time_before - time_after
            if c >= 0:
                c = 0
            else:
                c = -1

        # Recompensas adicionais por coletar moedas e derrotar inimigos
        coins_collected = info.get("coins", 0)
        enemies_defeated = info.get("enemies_defeated", 0)
        additional_reward = coins_collected * 1 + enemies_defeated * 1

        # Recompensas adicionais com base na eficiência do agente (mais rápido e menos etapas)
        remaining_time = info.get("remaining_time", 0)
        num_steps = info.get("num_steps", 0)
        time_reward = remaining_time * 1
        steps_reward = -num_steps * 1

        # Penaliza o agente por morrer (d)
        d = -15 if "life" in info and info["life"] < 1 else 0

        if info.get("flag_get", False):
            reward = 15
            self.rewards += reward
            self.logger.record("reward", reward)
            return True

        # Soma as recompensas e penalidades para calcular a recompensa total
        reward = v + c + d + additional_reward + time_reward + steps_reward

        # Salva a recompensa total no log
        self.rewards += reward
        self.logger.record("reward", reward)

        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'melhor_modelo_mario_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True


# Cria o ambiente do jogo
env = make_env()

# Diretórios onde seram salvos os modelos e logs
CHECKPOINT_DIR = './mario_model_v3/'
LOG_DIR = './logs_v2/'

# Crie o modelo PPO com a política MlpPolicy ou Use um já salvo
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=LOG_DIR, batch_size=128, learning_rate=0.0004, vf_coef=0.5, ent_coef=0.01, n_steps=1024)
## model = PPO.load('./mario_model_v3/melhor_modelo_mario_450000', env=env, tensorboard_log=LOG_DIR, batch_size=128, learning_rate=0.0005, vf_coef=0.5, ent_coef=0.01) ##2

# Declaração do treinamento com callback personalizado
custom_callback = CustomCallback(check_freq=50000, save_path=CHECKPOINT_DIR)
model.learn(total_timesteps=1000000, callback=custom_callback)

# Salva o modelo depois de concluir
model.save("mario_model")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./logs_v2/PPO_6
-----------------------------
| reward             | 0    |
| time/              |      |
|    fps             | 132  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 1024 |
-----------------------------
-----------------------------------------
| reward                  | 0           |
| time/                   |             |
|    fps                  | 53          |
|    iterations           | 2           |
|    time_elapsed         | 38          |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.008185441 |
|    clip_fraction        | 0.0359      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.6        |
|    explained_variance   | -0.00644    |
|    learning_rate        | 0.0004      |
|    loss                 | 90.3        |
|    n_u